# 0. Imports

In [ ]:
# Imports
from   IPython.display import display, HTML
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import logging
logging.basicConfig()
logging.getLogger('apscheduler').setLevel(logging.CRITICAL)

# Display options
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

In [ ]:
import mimicLOB as mlob

# 1. LOB Creation

In [ ]:
# b_tape = True means the LOB
LOB = mlob.OrderBook(tick_size = 0.1, 
                    b_tape    = True,
                    verbose   = False)

## Simulation configuration

#### Random Agent : a random agent is either a liquidity provider via limit order or a liquidity consumer via market orders. The agent has an intensity of sending order. 
#### Random agents react to exogenous news, their intensity of sending orders is shifted according to news arrival. 
#### News object is a random process between 0 & 100, 0 being the worst news and 100 very good news. For examples, buyers will buy more if the news are very good. This enables trends in prices.

#### Parameters :
1. distant : if true, the agent must have the server address. If False, he should have the orderbook object passed in the configuration.
2. server : when distant is True, the http address of the server the LOB is running on.
3. id : optional, default is generic. the if is used in the transaction tape.
4. b_record : optional, boolean, default is False. If True, the agent records all his activity : sent orders, executed trades, pending orders ...

In [ ]:
Simulationconfig = {'information' : {'basicInfo' : {'type'        : 'genericInformation',
                                                    'intensity'   : 0.03}},
                    'agents'      : {'agent1' : {'type'         : 'randomAgent',
                                                'subtype'       : 'randomLimitBuyer',
                                                 'id'           : 'agent1',
                                                'orderbook'     : LOB,
                                                'intensity'     : 0.4,
                                                'quantityRange' : [50, 200, 10]},
                                    'agent2' : {'type'          : 'randomAgent',
                                                'subtype'       : 'randomMarketBuyer',
                                                 'id'           : 'agent2',
                                                'orderbook'     : LOB,
                                                'intensity'     : 0.4,
                                                'quantityRange' : [50, 200, 10]},
                                    'agent3' : {'type'          : 'randomAgent',
                                                'subtype'       : 'randomLimitSeller',
                                                 'id'           : 'agent3',
                                                'orderbook'     : LOB,
                                                'intensity'     : 0.4,
                                                'quantityRange' : [50, 200, 10]},
                                    'agent4' : {'type'          : 'randomAgent',
                                                'subtype'       : 'randomMarketSeller',
                                                 'id'           : 'agent4',
                                                'orderbook'     : LOB,
                                                'intensity'     : 0.4,
                                                'quantityRange' : [50, 200, 10]},
                                    'MM'     : {'type'          : 'basicMM',
                                                 'id'           : 'MM',
                                                'orderbook'     : LOB,
                                                'refQuantity'   : 100,
                                                'refPrice'      : 100,
                                                'b_record'      : True}}}

## Create News Information

In [ ]:
# Channel (doubly linked chain of news)
newsChannel = mlob.Channel()

# keeps references on information sources
newsFactory = {}
config_news = Simulationconfig['information']
for info in config_news:
    config_news[info]['channel'] = newsChannel
    newsFactory[info] = mlob.utils.getInstance('mimicLOB.information', config_news[info])

## Create the agents
The agents is constantly connected to the distant LOB via the server address or the LOB address.

In [ ]:
agentFactory  = {} # keeps references on agents
config_agents = Simulationconfig['agents']

for agent in config_agents:
    config_agents[agent]['channel'] = newsChannel
    agentFactory[agent] = mlob.utils.getInstance('mimicLOB.agent', config_agents[agent])   
    
# The market (LOB admin)
market = mlob.genericAgent(id = 'market', orderbook = LOB)

## Launch Trading

#### We launch trading

#### You have to execute the stop or pause command to stop simulation (wait a few seconds of trading at least)

In [ ]:
from apscheduler.schedulers.background import BackgroundScheduler

sched = BackgroundScheduler()

def startSimulation():
    for agent in agentFactory:
        agentFactory[agent].start(sched)
        
    for news in newsFactory:
        newsFactory[news].start(sched)

In [ ]:
sched.start()
startSimulation()

## Pause Trading
#### INFO : trading still continues, even if the previous cell is not 'running'. The aps scheduler executes tasks in the background !

In [ ]:
try:
    sched.pause()
except Exception as e:
    print(e)

## Resume Trading

In [ ]:
try:
    sched.resume()
except Exception as e:
    print(e)

## Stop Trading

In [ ]:
try:
    sched.shutdown()
except Exception as e:
    print(e)

In [ ]:
# uncomment to reset lob : 
# market.resetLOB_PendingOrders()

## Get LOB

In [ ]:
LOBstate = market.getLOBState()
LOBstate = LOBstate.set_index('Price').sort_index()
LOBstate.plot.bar(figsize=(20, 7))
plt.show()

## Get Price Tape

In [ ]:
histoPrices = market.getPriceTape().astype(float)
histoPrices.plot(figsize=(20,7))

# OHLC
display(f'open  : {histoPrices.iloc[0,0]}')
display(f'high  : {histoPrices.max()[0]}')
display(f'low   : {histoPrices.min()[0]}')
display(f'close : {histoPrices.iloc[-1, 0]}')

plt.show()

## Get Transaction Tape

In [ ]:
TransactionTape = market.getTransactionTape()

In [ ]:
TransactionTape

## Get the MM book history

In [ ]:
executedTrades = pd.DataFrame.from_dict(agentFactory['MM'].executedtrades).T
executedTrades.set_index('time')

In [ ]:
# Book evolution :
book = mlob.utils.getBookHistory(executedTrades, 'MM')

In [ ]:
book.astype(float).plot(figsize=(25,7))